In [1]:
import spacy
from spacy import displacy

nlp = spacy.load('pt')

# Named Entity Recognition

Named Entities supported by Spacy: https://spacy.io/api/annotation#named-entities

In [2]:
doc = nlp('Maria está se mudando para Paris. No dia 01/02/2019 ela irá partir.')
for entidade in doc.ents:
    print(entidade.text, entidade.label_)

Maria PER
Paris LOC


In [3]:
displacy.render(doc, style='ent', jupyter=True)

In [4]:
spacy.explain('PER')

'Named person or family.'

In [5]:
# treinar o modelo com os novos dados
!python train_new_entity_type.py -m pt -o modelo_kodama.md

Loaded model 'pt'
Losses {'ner': 26.967837947885915}
Losses {'ner': 25.38339074434542}
Losses {'ner': 24.675209609953537}
Losses {'ner': 16.55306489537213}
Losses {'ner': 23.928378237463676}
Losses {'ner': 27.73959219991957}
Losses {'ner': 30.613730491642855}
Losses {'ner': 15.423950874644106}
Losses {'ner': 23.33050176337929}
Losses {'ner': 18.218481086660177}
Losses {'ner': 26.903662341763265}
Losses {'ner': 18.080315225175582}
Losses {'ner': 19.11660557515461}
Losses {'ner': 18.372059826680925}
Losses {'ner': 17.601169822155498}
Losses {'ner': 19.62259830746916}
Losses {'ner': 18.707242312542803}
Losses {'ner': 21.21788588963682}
Losses {'ner': 15.38832595932763}
Losses {'ner': 19.142946683394257}
Losses {'ner': 11.666053919229398}
Losses {'ner': 12.440184484443307}
Losses {'ner': 17.008277333545266}
Losses {'ner': 22.44728426736401}
Losses {'ner': 16.050916349978706}
Losses {'ner': 15.379875017108134}
Losses {'ner': 11.514700804189488}
Losses {'ner': 12.159411488981128}
Losses {'ne

In [6]:
nlp = spacy.load('./modelo_kodama.md')

In [7]:
doc = nlp('quero meu relatório de vendas entre 01/02/2020 e 01/03/2020')
displacy.render(doc, style='ent', jupyter=True)

Nós podemos observar que com um modelo probabislítico temos alguns problemas em identificar datas. Precisamos de mais exemplos e talvez nesse caso seja mais interessante utilizar uma abordagem baseda em regras (regex) e no caso de dados relativas como "ultimo mes" "semestre de 2020" podemos utilizar uma abordagem combinada com regras e bases de conhecimento

# Rule based and Knowledge Based

https://spacy.io/usage/rule-based-matching
https://spacy.io/usage/rule-based-matching#entityruler

In [8]:
from spacy.lang.pt import Portuguese
from spacy.pipeline import EntityRuler

nlp = Portuguese() # if you want to combina with existing prob model use spacy.load('pt')
ruler = EntityRuler(nlp)
patterns = [
    {
        'label': 'DATE', 'pattern': 
        [    # this will run per token and check the token sequence to match the rules
            {'TEXT' : {"REGEX": "[uú]ltimo"} },
            {'TEXT' : {"REGEX": "m[êe]s"} },
        ], 
        'id': 'date'
    },
    {
        'label': 'DATE', 'pattern': 
        [
            {'TEXT' : {"REGEX": "[uú]ltimo"} },
            {'TEXT' : {"REGEX": "\d"} },
            {'TEXT' : {"REGEX": "m[êe]s"} },
        ], 
        'id': 'date'
    },
    {
        'label': 'DATE', 'pattern': 
        [
            {'TEXT' : {"REGEX": "([Jj]aneiro|[Ff]evereiro|[Mm]ar[çc]o|[Aa]bril|[Mm]aio|[Jj]unho|[Jj]ulho|[Aa]gosto|[Ss]etembro|[Oo]utubro|[Nn]ovembro|[Dd]ezembro)"} }
        ], 
        'id': 'date'
    }, {
        'label': 'DATE', 'pattern': 
        [
            {'TEXT' : {"REGEX": "(hoje|ontem)"} }
        ], 
        'id': 'date'
    }
]
ruler.add_patterns(patterns)
nlp.add_pipe(ruler)

messages = [
    'quero relatório de vendas do ultimo mês',
    'como foram as minhas vendas nos últimos 3 meses',
    'vendas do mês de agosto',
    'minhas vendas do mes de março até junho',
    'como foram as minhas vendas entre Maio e Agosto',
    'quanto vendi entre abril e junho?',
    'qual produto eu mais vendi hoje'
]

def test_date_entity(messages):
    for message in messages:
        doc = nlp(message)
        print([(ent.text, ent.label_) for ent in doc.ents])

test_date_entity(messages)

# nlp.to_disk("/path/to/model") saving model

[('ultimo mês', 'DATE')]
[('últimos 3 meses', 'DATE')]
[('agosto', 'DATE')]
[('março', 'DATE'), ('junho', 'DATE')]
[('Maio', 'DATE'), ('Agosto', 'DATE')]
[('abril', 'DATE'), ('junho', 'DATE')]
[('hoje', 'DATE')]


# Dictionary based

In [9]:
from src.ner import NER

In [10]:
ner = NER(['parmegiana de frango', 'bife a parmegiana', 'refrigerante coca-cola', 'heineken 600ml'])
ner.vectorizer.vocabulary_
ner.recognize('quantas coca-colas tenho no estoque ?', algo='edit_distance')

[('coca colas', 'PRODUCT')]